# 1. 문제정의
- 영화리뷰 데이터셋을 활용해 긍정리뷰와 부정리뷰를 구분해보자
- 긍정/부정에서 자주 사용되는 단어를 확인해보자

# 2. 데이터 수집

In [43]:
from sklearn.datasets import load_files  # 여러개의 파일을 한번에 읽어들이는 함수

In [44]:
# 폴더지정해서 가져오기
train_data_url = 'data/aclImdb/train'
reviews_train = load_files(train_data_url, shuffle = True)

In [45]:
# esc 키 누르고 f 누르면 한번에 글바꾸는 replace 할 수 잇는 거 나옴
test_data_url = 'data/aclImdb/test'
reviews_test = load_files(test_data_url, shuffle = True)

In [46]:
reviews_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [47]:
len(reviews_train['data'])

25000

In [48]:
len(reviews_test['data'])

25000

In [49]:
reviews_train['target_names']

['neg', 'pos']

# 3. 데이터 전처리

In [50]:
reviews_train['data'][0]

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."

In [51]:
# 필요없는 br태그 제거하기

#원래는 
# replace - string안에서 내가원하는 string을 다른 문자열로 바꿔주는
# reviews_train['data'][0].replace("<br />"," ")

#근데 이거는 bite라는 새로운 자료형이라서 저게 안됌 - 그래서 bite라는걸 입력해줘야함
reviews_train['data'][0].replace(b"<br />",b" ")

b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.  It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see.   Flawed but honest with a terrible honesty."

#### 리스트 내포

In [52]:
#q 비어있는 리스트하나만들고 가지고있는 데이터수만큼 for문 돌려서 데이터수만큼 다시 저장하도록

In [53]:
text_train = [txt.replace(b"<br />",b" ") for txt in reviews_train['data']]
text_test = [txt.replace(b"<br />",b" ") for txt in reviews_test['data']]

In [54]:
# 위에꺼 풀어쓰면
text_train = []

for txt in reviews_train['data']:
    text_train.append(txt.replace(b"<br />",b" "))
    #append 추가 - br태그를 공백으로 바꾼것들을 추가해주고 변수에 저장

In [55]:
# 25000개를 다 전처리하려면 오래걸리닌까 데이터 쪼개놓고 가기 ( 전체 말고 1000개만 분석 및 훈련시켜보자)
text_train = text_train[:1000]
text_test = text_test[:1000]

## 토큰화 
- 글자를 수치화 

#### BOW (bag of words ) - 토큰화 방법 중 하나

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
text = ['맞춤법 검사 및 상황에 맞는 사자성어,속담 추천 서비스',
        '모션 감지를 통한 음악 추천 및 스트리밍 서비스',
        '초보자를 위한 제사상가이드',
        '음성데이터를 통한 음악 분류 및 추천 서비스'
       ]

In [58]:
text_cv = CountVectorizer()
# bow방식을 카운터백터 로 실행시킨거

In [59]:
text_cv.fit(text) #수치화하려면 텍스트데이터를 어떤걸 어떻게 수치화할건지 학습하는 과정

CountVectorizer()

In [60]:
text_cv.vocabulary_  # 단어사전 구축 - 텍스트에서 사용한 값들 중에 한번이라도 나온 단어들이 다 나옴. 
# 띄어쓰기를 기준으로 하나의 단어라고 생각하고 
# 중복된건 하나만 나타내고
# 사용된 모든 단어들을 정리해옴

{'맞춤법': 3,
 '검사': 1,
 '상황에': 7,
 '맞는': 2,
 '사자성어': 6,
 '속담': 9,
 '추천': 16,
 '서비스': 8,
 '모션': 4,
 '감지를': 0,
 '통한': 17,
 '음악': 13,
 '스트리밍': 10,
 '초보자를': 15,
 '위한': 11,
 '제사상가이드': 14,
 '음성데이터를': 12,
 '분류': 5}

In [61]:
text_cv.transform(text).toarray()

# 0번단어가 없고 , 1번단어가 있고, 2번단어가 있고 ...
# 0은 없고 1은 있다는 수치화
# 안에 들어가있는 단어는 없다 0 , 있는 단어는 1 



array([[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1]],
      dtype=int64)

### 실제 데이터 적용

In [62]:
movie_count = CountVectorizer()
movie_count.fit(text_train) # 토큰화

CountVectorizer()

In [63]:
len(movie_count.vocabulary_)

17994

In [64]:
X_train = movie_count.transform(text_train)

In [65]:
X_test = movie_count.transform(text_test)

In [66]:
import numpy as np

In [67]:
np.bincount(X_train.toarray()[0]) # 숫자 갯수 세주는 함수

array([17903,    68,    15,     4,     3,     1], dtype=int64)

In [68]:
y_train = reviews_train['target'][:1000]
y_test = reviews_test['target'][:1000]

## 모델 선택 및 하이퍼 파라미터 튜닝

### 교차 검증 사용
- 모델 선택시 활용

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

In [70]:
tree = DecisionTreeClassifier()
svm = LinearSVC()

In [71]:
# 어떤 모델이 잘 어울리는지 확인해보기 위해서 교차 검증 - 그래서 하이퍼파라미터는 적용해보지 않을거

In [72]:
from sklearn.model_selection import cross_val_score
cross_val_score(tree, X_train, y_train, cv = 5).mean()

0.697

In [73]:
cross_val_score(svm, X_train, y_train, cv = 5).mean()

0.782

# 학습

In [74]:
svm.fit(X_train, y_train)

LinearSVC()

# 평가

In [75]:
svm.score(X_train, y_train)

1.0

In [76]:
svm.score(X_test, y_test)

0.777

# 예측

In [77]:
reviews = ["Wow. This is one of the most mind bending things in media. It makes Age Of Ultron better, it is very funny, the characters are so suprising and it also plays well into the MCU in other ways. The episode before the final one has great writing and the final showdowns are like watching an MCU film. This whole thing would be great as an MCU film, the way it plays. The return of people blipped also was interesting here and the story of Infinity War playing into the Vision storyline and how Wanda really just wanted more time with him. It is defismtly emotional and touching. Disney+ started off right with their first MCU tie in. The moral of this is that it seems to expose witchcraft rather then embrace it, which is good. (It might be exposing it). Jesus is our only hope.",
            """ "We are an unusual couple, you know." "Oh, I don't think that was ever in question." Now, before I begin, there are many opinions of this show. Many fans and people just introduced to the MCU have strong opinions which has made this a very divided addition to the MCU. Critically, it's been praised and there aren't many people who don't like it. But with the unique approach, some have been upset for such the high ratings. What I'm trying to say is I ask you to bear with me and respect my opinion. If you differ with it, there's nothing you can do to change my mind. The absolute masterpiece that is WandaVision blends the style of classic sitcoms with the MCU, in which Wanda Maximoff and Vision - two super-powered beings living their ideal suburban lives - begin to suspect that everything is not as it seems. From that teaser at the Super Bowl (I believe) over a year ago, this had been something exciting to look forward to. Literally no one knew what this would be about going in, really. I refrained from all trailers to go in as blind as possible. What I got was more than what I could imagine. Trying something new is what I've hoped for in the MCU for a while. As much as I do love a good majority of their movies, there's a fixed setup for almost all. WandaVision is something totally different and just what we needed. Had this been a show outside of the MCU, I think it would still be brilliant. Each episode is designed to go through a different era of television. How they handled it worked incredibly well. There's not a way of trying to modernize these old sitcoms from the 1950s and so forth, so it's like a replication of classic television. The sets, costumes, and camera lens and moment are amazing at doing so. One of my favorite games while watching is trying to see what show this episode was mainly based on. Accomplishing this couldn't have been easy and I applaud them for how they managed to get the feel of each era perfectly. At the heart of this story are amazing performances. Elizabeth Olson hasn't had such an amazing display of acting since Martha Marcy May Marlene. So much emotion is put forth into Wanda that she is by far the most developed character in the MCU even if she came in late. There are some really heavy scenes and she portrayed those flawlessly. It doesn't feel much like watching a magical being, but we understand she's just another person in this world. Scene 8 showcases it all. Alongside her is Paul Bettany. With his character of Vision — a literal computer-god-being — it's hard to get complete range. He has such a good start with the series by delivering comedy. He's really funny in it (and Olsen too). Once the mystery starts to unfold, he builds more and more character until episode 5 when he unleashes his full capabilities. Never has Vision felt so human before. I'd also like to highlight Kathryn Hahn, because she's such an amazing actress. Playing the nosy-neighbor of Agnes must've been such a fun time. She takes up all the screen time she can get. I won't spoil a thing, but later on in the series she gets her moment — literally everyone's favorite moment — and you just love her even more. Teyonah Paris, Kat Dennings, and Randall Park also do well with their supporting roles. This is an ensemble piece for sure and the way they work off of each other shows dedication. It's hard to discuss so much without spoiling because certain things do need to be addressed, but I shall refrain. Story is where people have been divided. Some thought it took way too long to get into, and I just don't see why. It's a series, not a movie, so setup is much different. And if it were to have moved on quickly, the mystery element would've lost its momentum. Every episode has some sort of question leading up to the finale. That's where fan theories came in and caused even more disappointments. I support fan theories, but I don't base my expectation on the rest of the show. Even if I had some hopeful thoughts, I never expected them to show up later on. With expectation, you can only be disappointed. My suggestion is to not have anything in mind when going in. Questions kept building and that's what made this the most gripping show I had seen in such a long time. I would stay up till 2 a.m. for the release of the new episodes because I just had to know what would come next. With a series, there was more time to develop and think about plot and character. Most importantly, though, there was enough time to build upon the past episodes and make an enjoyable time for both the sitcom moments and the Marvel storyline. Sure, not every episode is as great as the one before or after. I don't think you'll ever find a show with each episode being perfect as ever. Take a highly regarded show like Breaking Bad. Many think of it as a perfect show, but it's not like they think every episode is perfect. To quote Steven Universe, one of my favorite shows, "if every pork chop were perfect, we wouldn't have hotdogs." If every episode were perfect, it wouldn't have that range that it has. I don't grade a show based on each episode, but rather as a whole. And throughout the duration of WandaVision, I had an absolute blast. Marvel, Matt Shakman, and the whole crew made something unforgettable. I believe this to be the greatest thing the MCU has given us. There will never be a show quite like WandaVision. Only a few shows have gotten my perfect rating, and this ranks among them. """ ,
          ''' ( SPOILERS) Absolute garbage and a waste of time. Full of plot twists that end up being nothing. Vision having holes in his body had nothing to do with the plot. Pietro having holes in his body had nothing to do with the plot. Pietro being from X'men was just a random coincidence. Also, every time a new male character walked into the show you knew he was either a wimp or evil. They even made pietros real last name "bohner" to make fun of manhood. Imagine if a female character everyone was stoked on turned out to be some random lady named "Vachina". Also, the physical vision just flew off for no reason, and digital vision never decided to tell wanda about his existence. Why? Lazy writing. Additionally at the end rhambeaou tells wanda "they will never know what you sacrificed". What the heck?! Like maybe apologize for trapping and tormenting these people every day for like a month. How on earth is wanda the victim or the "good-guy" in this show. She is literally a villain causing everyone pain, but it is "ok" because she did it out of a place of pain. Im sorry, almost all villains do evil out of a place of pain, that doesnt make it ok. Stupid, sexist show with bad plot that treats its audience like idiots. '''
          ]

In [78]:
# 예측 데이터도 학습할때와 동일한 전처리를 진행
# 우리가 진행한 전처리는 br태그 제거, BOW 토근화 및 수치화 

In [79]:
reviews = [txt.replace("<br />"," ") for txt in reviews]

In [80]:
reviews = movie_count.transform(reviews)

In [81]:
reviews

<3x17994 sparse matrix of type '<class 'numpy.int64'>'
	with 602 stored elements in Compressed Sparse Row format>

In [82]:
svm.predict(reviews)

array([1, 1, 0])

## 어떤 단어가 긍정/ 부정에 많은 영향을 줬는지 알아보자

In [86]:
movie_count.vocabulary_ # 학습에 사용된 모든 단어

{'zero': 17969,
 'day': 4144,
 'leads': 9204,
 'you': 17932,
 'to': 16268,
 'think': 16118,
 'even': 5610,
 're': 12875,
 'why': 17634,
 'two': 16652,
 'boys': 2085,
 'young': 17933,
 'men': 10134,
 'would': 17815,
 'do': 4800,
 'what': 17573,
 'they': 16107,
 'did': 4545,
 'commit': 3323,
 'mutual': 10681,
 'suicide': 15573,
 'via': 17191,
 'slaughtering': 14667,
 'their': 16077,
 'classmates': 3058,
 'it': 8524,
 'captures': 2532,
 'must': 10666,
 'be': 1533,
 'beyond': 1746,
 'bizarre': 1825,
 'mode': 10397,
 'of': 11163,
 'being': 1638,
 'for': 6382,
 'humans': 7840,
 'who': 17619,
 'have': 7408,
 'decided': 4199,
 'withdraw': 17730,
 'from': 6552,
 'common': 3329,
 'civility': 3027,
 'in': 8074,
 'order': 11292,
 'define': 4250,
 'own': 11448,
 'world': 17793,
 'coupled': 3756,
 'destruction': 4453,
 'is': 8504,
 'not': 11014,
 'perfect': 11751,
 'movie': 10588,
 'but': 2356,
 'given': 6857,
 'money': 10446,
 'time': 16233,
 'the': 16068,
 'filmmaker': 6141,
 'and': 827,
 'actors'

In [87]:
svm.coef_ # 각 단어가 갖는 가중치(중요도 중요도가 높으면 긍정에, 낮으면 부정에 영향을 많이끼치는)

array([[-0.0070644 , -0.00820312, -0.01605669, ...,  0.        ,
        -0.00186803,  0.        ]])

In [90]:
import pandas as  pd
# 사용된 단어 데이터 프레임으로 정리하기 
df = pd.DataFrame([movie_count.vocabulary_.keys(),
             movie_count.vocabulary_.values()])
df

,0,1,2,3,4,5,6,7,8,9,...,17984,17985,17986,17987,17988,17989,17990,17991,17992,17993
0,zero,day,leads,you,to,think,even,re,why,two,...,incorporation,humankind,vistas,unrushed,preconceptions,wrought,imposing,amplified,hitherto,unproven
1,17969,4144,9204,17932,16268,16118,5610,12875,17634,16652,...,8118,7837,17292,16956,12278,17857,8050,792,7644,16935


In [91]:
# 위에서 한 정리가 엄청 깔끔하진않아서 다시한번 바꿔주기
df = df.T  # 전치 (행과 열 바꿈)
df

,0,1
0,zero,17969
1,day,4144
2,leads,9204
3,you,17932
4,to,16268
...,...,...
17989,wrought,17857
17990,imposing,8050
17991,amplified,792
17992,hitherto,7644


In [92]:
# 1번 컬럼(인덱스번호)를 기준으로 정렬
df.sort_values(by=1,inplace = True)
df

,0,1
8649,00,0
11236,000,1
13401,007,2
9679,01,3
16687,06,4
...,...,...
12588,zorro,17989
16886,zubeidaa,17990
17630,zuzz,17991
9848,zzzzz,17992


In [94]:
# 2차원 데이터 1차원으로 변환후 (reshape)
# coef 컬럼 만들어서 값 삽입

df['coef'] = svm.coef_.reshape(-1)
df
# 양수로 값이 클수록 양성에 영향을 많이끼친거
# 값이 작을수록 음성에 영향을 많이 끼친거 ? 

,0,1,coef
8649,00,0,-0.007064
11236,000,1,-0.008203
13401,007,2,-0.016057
9679,01,3,0.000000
16687,06,4,0.000000
...,...,...,...
12588,zorro,17989,0.045448
16886,zubeidaa,17990,0.000000
17630,zuzz,17991,0.000000
9848,zzzzz,17992,-0.001868


In [98]:
# 긍정 / 부정에 영향을 끼친 글자들을 확인하기 위해
# coef컬럼을 기준으로 정렬
df.sort_values(by = 'coef',inplace= True)
df


,0,1,coef
95,bad,1376,-0.296184
454,worst,17810,-0.277235
691,waste,17446,-0.222575
1828,awful,1331,-0.186486
144,instead,8315,-0.174227
...,...,...,...
1188,enjoy,5413,0.167843
317,still,15303,0.200493
1236,feel,6036,0.201511
52,perfect,11751,0.203336


# 토큰화 + 모델학습 한번에 진행하기

In [97]:
# 기능을 연결하는 함수
# 연결 순서는 실행순서와 맞게 
from sklearn.pipeline import make_pipeline

In [99]:
pipe_model = make_pipeline(CountVectorizer(),LinearSVC())

In [100]:
pipe_model.fit(text_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('linearsvc', LinearSVC())])

In [104]:
pipe_model.predict(reviews)

array([1, 1, 0])

# 파이프라인에 GridSearch 사용하기

In [ ]:
# countvectorizer에서 사용하는 하이퍼 파라미터
## min_df : 문서에서 등장하는 최소 빈도수 
## max_df : 문서에서 등장하는 최대 빈도수
## ngram_range = (1,3) : 1~ 3 gram까지의 토큰화를 사용

#svm
## C = 규제 : 값이 클수록 작은 규제

In [110]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'countvectorizer__max_df' : [1000,2000,3000],
    'countvectorizer__min_df' : [10,20,30],
    'countvectorizer__ngram_range' : [(1,1),(1,2),(1,3)],
    'linearsvc__C' : [0.01 , 0.1 , 1 , 10 , 100]
}
#최대 10번이상 ,20번이상, 30번이상 등장한 단어들만 단어사전에 등록을 하겠다

In [111]:
grid = GridSearchCV(pipe_model, param_grid, cv = 5)
grid.fit(text_train, y_train)

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\smhrd\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('linearsvc', LinearSVC())]),
             param_grid={'countvectorizer__max_df': [1000, 2000, 3000],
                         'countvectorizer__min_df': [10, 20, 30],
                         'countvectorizer__ngram_range': [(1, 1), (1, 2),
                                                          (1, 3)],
                         'linearsvc__C': [0.01, 0.1, 1, 10, 100]})

In [112]:
grid.best_params_

{'countvectorizer__max_df': 1000,
 'countvectorizer__min_df': 10,
 'countvectorizer__ngram_range': (1, 3),
 'linearsvc__C': 0.01}

In [113]:
grid.best_score_

0.7849999999999999

# 예측

In [103]:
reviews = ["Wow. This is one of the most mind bending things in media. It makes Age Of Ultron better, it is very funny, the characters are so suprising and it also plays well into the MCU in other ways. The episode before the final one has great writing and the final showdowns are like watching an MCU film. This whole thing would be great as an MCU film, the way it plays. The return of people blipped also was interesting here and the story of Infinity War playing into the Vision storyline and how Wanda really just wanted more time with him. It is defismtly emotional and touching. Disney+ started off right with their first MCU tie in. The moral of this is that it seems to expose witchcraft rather then embrace it, which is good. (It might be exposing it). Jesus is our only hope.",
            """ "We are an unusual couple, you know." "Oh, I don't think that was ever in question." Now, before I begin, there are many opinions of this show. Many fans and people just introduced to the MCU have strong opinions which has made this a very divided addition to the MCU. Critically, it's been praised and there aren't many people who don't like it. But with the unique approach, some have been upset for such the high ratings. What I'm trying to say is I ask you to bear with me and respect my opinion. If you differ with it, there's nothing you can do to change my mind. The absolute masterpiece that is WandaVision blends the style of classic sitcoms with the MCU, in which Wanda Maximoff and Vision - two super-powered beings living their ideal suburban lives - begin to suspect that everything is not as it seems. From that teaser at the Super Bowl (I believe) over a year ago, this had been something exciting to look forward to. Literally no one knew what this would be about going in, really. I refrained from all trailers to go in as blind as possible. What I got was more than what I could imagine. Trying something new is what I've hoped for in the MCU for a while. As much as I do love a good majority of their movies, there's a fixed setup for almost all. WandaVision is something totally different and just what we needed. Had this been a show outside of the MCU, I think it would still be brilliant. Each episode is designed to go through a different era of television. How they handled it worked incredibly well. There's not a way of trying to modernize these old sitcoms from the 1950s and so forth, so it's like a replication of classic television. The sets, costumes, and camera lens and moment are amazing at doing so. One of my favorite games while watching is trying to see what show this episode was mainly based on. Accomplishing this couldn't have been easy and I applaud them for how they managed to get the feel of each era perfectly. At the heart of this story are amazing performances. Elizabeth Olson hasn't had such an amazing display of acting since Martha Marcy May Marlene. So much emotion is put forth into Wanda that she is by far the most developed character in the MCU even if she came in late. There are some really heavy scenes and she portrayed those flawlessly. It doesn't feel much like watching a magical being, but we understand she's just another person in this world. Scene 8 showcases it all. Alongside her is Paul Bettany. With his character of Vision — a literal computer-god-being — it's hard to get complete range. He has such a good start with the series by delivering comedy. He's really funny in it (and Olsen too). Once the mystery starts to unfold, he builds more and more character until episode 5 when he unleashes his full capabilities. Never has Vision felt so human before. I'd also like to highlight Kathryn Hahn, because she's such an amazing actress. Playing the nosy-neighbor of Agnes must've been such a fun time. She takes up all the screen time she can get. I won't spoil a thing, but later on in the series she gets her moment — literally everyone's favorite moment — and you just love her even more. Teyonah Paris, Kat Dennings, and Randall Park also do well with their supporting roles. This is an ensemble piece for sure and the way they work off of each other shows dedication. It's hard to discuss so much without spoiling because certain things do need to be addressed, but I shall refrain. Story is where people have been divided. Some thought it took way too long to get into, and I just don't see why. It's a series, not a movie, so setup is much different. And if it were to have moved on quickly, the mystery element would've lost its momentum. Every episode has some sort of question leading up to the finale. That's where fan theories came in and caused even more disappointments. I support fan theories, but I don't base my expectation on the rest of the show. Even if I had some hopeful thoughts, I never expected them to show up later on. With expectation, you can only be disappointed. My suggestion is to not have anything in mind when going in. Questions kept building and that's what made this the most gripping show I had seen in such a long time. I would stay up till 2 a.m. for the release of the new episodes because I just had to know what would come next. With a series, there was more time to develop and think about plot and character. Most importantly, though, there was enough time to build upon the past episodes and make an enjoyable time for both the sitcom moments and the Marvel storyline. Sure, not every episode is as great as the one before or after. I don't think you'll ever find a show with each episode being perfect as ever. Take a highly regarded show like Breaking Bad. Many think of it as a perfect show, but it's not like they think every episode is perfect. To quote Steven Universe, one of my favorite shows, "if every pork chop were perfect, we wouldn't have hotdogs." If every episode were perfect, it wouldn't have that range that it has. I don't grade a show based on each episode, but rather as a whole. And throughout the duration of WandaVision, I had an absolute blast. Marvel, Matt Shakman, and the whole crew made something unforgettable. I believe this to be the greatest thing the MCU has given us. There will never be a show quite like WandaVision. Only a few shows have gotten my perfect rating, and this ranks among them. """ ,
          ''' ( SPOILERS) Absolute garbage and a waste of time. Full of plot twists that end up being nothing. Vision having holes in his body had nothing to do with the plot. Pietro having holes in his body had nothing to do with the plot. Pietro being from X'men was just a random coincidence. Also, every time a new male character walked into the show you knew he was either a wimp or evil. They even made pietros real last name "bohner" to make fun of manhood. Imagine if a female character everyone was stoked on turned out to be some random lady named "Vachina". Also, the physical vision just flew off for no reason, and digital vision never decided to tell wanda about his existence. Why? Lazy writing. Additionally at the end rhambeaou tells wanda "they will never know what you sacrificed". What the heck?! Like maybe apologize for trapping and tormenting these people every day for like a month. How on earth is wanda the victim or the "good-guy" in this show. She is literally a villain causing everyone pain, but it is "ok" because she did it out of a place of pain. Im sorry, almost all villains do evil out of a place of pain, that doesnt make it ok. Stupid, sexist show with bad plot that treats its audience like idiots. '''
          ]